# Lesson 10
# Peter Lorenz

## 0. Preparation
Import libraries:

In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

Set global options:

In [3]:
# Display multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Suppress scientific notation
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)

Set constants:

In [12]:
# Declare constants
EMBEDDING_VECOR_LENGTH = 250
NUM_WORDS = 5000
MAX_LEN = 500
MAX_REVIEW_LENGTH = 300

## 1. Read data set into training and testing
In this section we read the Reuters data set into training and test data sets:

In [5]:
# Deal with pickle bug
# See https://stackoverflow.com/questions/57176714/how-to-fix-error-when-load-dataset-in-keras
old = np.load
np.load = lambda *a,**k: old(*a,allow_pickle=True)

# Load data set
(X_train, y_train), (X_test, y_test) = \
    tf.keras.datasets.reuters.load_data(
        path='reuters.npz', num_words=NUM_WORDS, skip_top=10, maxlen=MAX_LEN, 
        test_split=0.2, seed=1)

# Restore numpy load
np.load = old
del(old)

Examine the shape of the data:

In [6]:
# Examine data
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(8621,)

(2156,)

(8621,)

(2156,)

## 2. Prepare the data set
In this section we prepare the data set. We begin by padding the sequences in the training and test data:

In [9]:
# Pad data
x_train_padded = pad_sequences(X_train, maxlen = MAX_REVIEW_LENGTH)
x_test_padded = pad_sequences(X_test, maxlen = MAX_REVIEW_LENGTH)

Next we verify the shape of the data:

In [11]:
# Display shape
x_train_padded.shape
x_test_padded.shape

(8621, 300)

(2156, 300)

As a sanity check we examine the padded data itself:

In [10]:
x_train_padded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

With the data prepared, we can now build our models.

## 3. Build and compile 3 different models
In this section we build and compile 3 different models using Keras LTSM, ideally improving model at each iteration.

In [15]:
# Build model
model = keras.models.Sequential()
model.add(keras.layers.Embedding(NUM_WORDS, EMBEDDING_VECOR_LENGTH, input_length = MAX_REVIEW_LENGTH))
model.add(keras.layers.LSTM(32))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 250)          1250000   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                36224     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,286,257
Trainable params: 1,286,257
Non-trainable params: 0
_________________________________________________________________
None


Now we train our model:

In [16]:
model.fit(x_train_padded, y_train, validation_data = (x_test_padded, y_test), 
          epochs = 2, batch_size = 128)

Train on 8621 samples, validate on 2156 samples
Epoch 1/2
8621/8621 [==============================] - 55s 6ms/sample - loss: -38.4447 - acc: 0.0454 - val_loss: -73.2146 - val_acc: 0.0533
Epoch 2/2
8621/8621 [==============================] - 53s 6ms/sample - loss: -88.2862 - acc: 0.0455 - val_loss: -95.1879 - val_acc: 0.0533


With the model trained, we check the accuracy:

In [17]:
# Evaluate model
scores = model.evaluate(x_test_padded, y_test)
print("Accuracy: {:.2f}%".format(scores[1]*100))

2156/2156 [==============================] - 5s 2ms/sample - loss: -95.1879 - acc: 0.0533
Accuracy: 5.33%


The accuracy of our initial model is rather poor with just 5% accuracy.

## 4. Describe and explain your findings
In this assignment ...